In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
from keras.models import Model
from tensorflow import keras
import pandas as pd
from os import walk, listdir
import os 

In [2]:
categories = pd.read_csv("category_mapping.csv", sep=";")

In [3]:
categories["ID_str"] = categories["ID"].apply(lambda x: str(x).zfill(3))

In [4]:
categories_to_ID = dict(zip(categories["ID_str"],categories["Name"]))

In [5]:
data_path = "./data/train_frames"
output_path = "./data/train_cnn_features_pretrained_v2/"

video_categories = listdir(data_path)
image_files = []
for category in video_categories:
    category_path = data_path+"/"+category
    image_files_category = listdir(category_path)
    image_files = image_files+image_files_category

In [6]:
len(video_categories)

64

In [7]:
dataset = pd.DataFrame()

In [8]:
dataset["frames_files"] = image_files

In [9]:
dataset["category_ID"] = dataset["frames_files"].apply(lambda x: x[:3])

In [10]:
dataset["category_name"] = dataset["category_ID"].apply(lambda x: categories_to_ID[x])

In [11]:
dataset["person_ID"] = dataset["frames_files"].apply(lambda x: x[4:7])
dataset["person_ID"] = dataset["frames_files"].apply(lambda x: x[4:7])

In [12]:
dataset["video_ID"] = dataset["frames_files"].apply(lambda x: x[:11])

In [13]:
dataset["video_path"] = data_path+"/"+dataset["category_ID"]+"/"+dataset["frames_files"]

In [14]:
dataset["frame_ID"] = dataset["frames_files"].apply(lambda x: int(x.split("frame_")[1].split(".")[0]))

In [15]:
MAX_SEQ_LENGTH = dataset["frame_ID"].max()+1
print(MAX_SEQ_LENGTH)

242


# Predict 

In [16]:
# CLASS_NAMES =["Sweet milk", "Argentina", "Barbecue", "Thanks"]
# MODEL_PATH = "./trained_models/signs_model_efficientnetv2-s_v2"


In [17]:
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [18]:
# def load_model(model_path):
#     model_keras = tf.keras.models.load_model(model_path)
#     return model_keras


# model = load_model(MODEL_PATH)

In [19]:
XX = model.input 
YY = model.layers[0].output
model_cnn_features = Model(XX, YY)

In [20]:
width = model.input_shape[1]
height = model.input_shape[2]

In [25]:
dataset_tf = tf.keras.preprocessing.image_dataset_from_directory(data_path,
                                                                 image_size=(width, height), batch_size=4)

Found 399000 files belonging to 64 classes.


In [26]:
dataset_predictions = model_cnn_features.predict(dataset_tf) 

ResourceExhaustedError: Graph execution error:

SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:GPU:0;e626acdc9f6dc591;/job:localhost/replica:0/task:0/device:CPU:0;edge_8_Identity;0:0
	 [[{{node Identity/_8}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_3143]

In [ ]:
dataset_predictions.shape

In [24]:
total_videos = len(dataset["video_ID"].unique())
i=1
for video_ID in dataset["video_ID"].unique():
    video_frames = dataset[dataset["video_ID"]==video_ID]
    category_ID = video_frames["category_ID"].unique()[0]
    number_of_frames_in_video = len(video_frames["frames_files"].unique())
    path_to_video = output_path+category_ID+"/"
    dataset_video_index = list(dataset[dataset["video_ID"]==video_ID].index)
    print(f"Predicting video: {i}/{total_videos} length: {len(dataset_video_index)}" )
    predictions_video = dataset_predictions[dataset_video_index] 
    zeros_to_pad = MAX_SEQ_LENGTH-predictions_video.shape[0]
    predictions_video = np.pad(predictions_video,  [(0, zeros_to_pad), (0, 0)])
    if not os.path.exists(path_to_video):
        os.makedirs(path_to_video)
    np.savez_compressed(path_to_video+video_ID+'.npz', predictions_video)
    i+=1 

Predicting video: 1/3120 length: 88
Predicting video: 2/3120 length: 148
Predicting video: 3/3120 length: 118
Predicting video: 4/3120 length: 118
Predicting video: 5/3120 length: 88
Predicting video: 6/3120 length: 118
Predicting video: 7/3120 length: 118
Predicting video: 8/3120 length: 118
Predicting video: 9/3120 length: 118
Predicting video: 10/3120 length: 118
Predicting video: 11/3120 length: 118
Predicting video: 12/3120 length: 118
Predicting video: 13/3120 length: 118
Predicting video: 14/3120 length: 118
Predicting video: 15/3120 length: 118
Predicting video: 16/3120 length: 148
Predicting video: 17/3120 length: 148
Predicting video: 18/3120 length: 148
Predicting video: 19/3120 length: 148
Predicting video: 20/3120 length: 118
Predicting video: 21/3120 length: 118
Predicting video: 22/3120 length: 118
Predicting video: 23/3120 length: 118
Predicting video: 24/3120 length: 148
Predicting video: 25/3120 length: 118
Predicting video: 26/3120 length: 118
Predicting video: 27/31

Predicting video: 219/3120 length: 118
Predicting video: 220/3120 length: 148
Predicting video: 221/3120 length: 148
Predicting video: 222/3120 length: 118
Predicting video: 223/3120 length: 148
Predicting video: 224/3120 length: 148
Predicting video: 225/3120 length: 148
Predicting video: 226/3120 length: 148
Predicting video: 227/3120 length: 118
Predicting video: 228/3120 length: 148
Predicting video: 229/3120 length: 148
Predicting video: 230/3120 length: 148
Predicting video: 231/3120 length: 118
Predicting video: 232/3120 length: 88
Predicting video: 233/3120 length: 88
Predicting video: 234/3120 length: 88
Predicting video: 235/3120 length: 118
Predicting video: 236/3120 length: 148
Predicting video: 237/3120 length: 148
Predicting video: 238/3120 length: 148
Predicting video: 239/3120 length: 148
Predicting video: 240/3120 length: 118
Predicting video: 241/3120 length: 118
Predicting video: 242/3120 length: 118
Predicting video: 243/3120 length: 118
Predicting video: 244/3120 l

Predicting video: 431/3120 length: 118
Predicting video: 432/3120 length: 118
Predicting video: 433/3120 length: 118
Predicting video: 434/3120 length: 118
Predicting video: 435/3120 length: 118
Predicting video: 436/3120 length: 118
Predicting video: 437/3120 length: 118
Predicting video: 438/3120 length: 118
Predicting video: 439/3120 length: 118
Predicting video: 440/3120 length: 118
Predicting video: 441/3120 length: 178
Predicting video: 442/3120 length: 118
Predicting video: 443/3120 length: 118
Predicting video: 444/3120 length: 118
Predicting video: 445/3120 length: 88
Predicting video: 446/3120 length: 118
Predicting video: 447/3120 length: 118
Predicting video: 448/3120 length: 118
Predicting video: 449/3120 length: 118
Predicting video: 450/3120 length: 118
Predicting video: 451/3120 length: 118
Predicting video: 452/3120 length: 118
Predicting video: 453/3120 length: 88
Predicting video: 454/3120 length: 88
Predicting video: 455/3120 length: 88
Predicting video: 456/3120 le

Predicting video: 645/3120 length: 88
Predicting video: 646/3120 length: 118
Predicting video: 647/3120 length: 88
Predicting video: 648/3120 length: 88
Predicting video: 649/3120 length: 88
Predicting video: 650/3120 length: 88
Predicting video: 651/3120 length: 118
Predicting video: 652/3120 length: 88
Predicting video: 653/3120 length: 88
Predicting video: 654/3120 length: 118
Predicting video: 655/3120 length: 118
Predicting video: 656/3120 length: 88
Predicting video: 657/3120 length: 118
Predicting video: 658/3120 length: 88
Predicting video: 659/3120 length: 88
Predicting video: 660/3120 length: 88
Predicting video: 661/3120 length: 118
Predicting video: 662/3120 length: 118
Predicting video: 663/3120 length: 88
Predicting video: 664/3120 length: 88
Predicting video: 665/3120 length: 118
Predicting video: 666/3120 length: 118
Predicting video: 667/3120 length: 88
Predicting video: 668/3120 length: 118
Predicting video: 669/3120 length: 118
Predicting video: 670/3120 length: 118


Predicting video: 858/3120 length: 118
Predicting video: 859/3120 length: 118
Predicting video: 860/3120 length: 118
Predicting video: 861/3120 length: 118
Predicting video: 862/3120 length: 148
Predicting video: 863/3120 length: 148
Predicting video: 864/3120 length: 148
Predicting video: 865/3120 length: 148
Predicting video: 866/3120 length: 118
Predicting video: 867/3120 length: 118
Predicting video: 868/3120 length: 118
Predicting video: 869/3120 length: 118
Predicting video: 870/3120 length: 118
Predicting video: 871/3120 length: 88
Predicting video: 872/3120 length: 118
Predicting video: 873/3120 length: 118
Predicting video: 874/3120 length: 118
Predicting video: 875/3120 length: 118
Predicting video: 876/3120 length: 88
Predicting video: 877/3120 length: 118
Predicting video: 878/3120 length: 148
Predicting video: 879/3120 length: 118
Predicting video: 880/3120 length: 118
Predicting video: 881/3120 length: 118
Predicting video: 882/3120 length: 118
Predicting video: 883/3120 

...

Predicting video: 2103/3120 length: 92
Predicting video: 2104/3120 length: 122
Predicting video: 2105/3120 length: 122
Predicting video: 2106/3120 length: 122
Predicting video: 2107/3120 length: 152
Predicting video: 2108/3120 length: 122
Predicting video: 2109/3120 length: 122
Predicting video: 2110/3120 length: 122
Predicting video: 2111/3120 length: 152
Predicting video: 2112/3120 length: 122
Predicting video: 2113/3120 length: 152
Predicting video: 2114/3120 length: 122
Predicting video: 2115/3120 length: 122
Predicting video: 2116/3120 length: 152
Predicting video: 2117/3120 length: 122
Predicting video: 2118/3120 length: 122
Predicting video: 2119/3120 length: 122
Predicting video: 2120/3120 length: 122
Predicting video: 2121/3120 length: 152
Predicting video: 2122/3120 length: 122
Predicting video: 2123/3120 length: 152
Predicting video: 2124/3120 length: 152
Predicting video: 2125/3120 length: 122
Predicting video: 2126/3120 length: 122
Predicting video: 2127/3120 length: 122
P

Predicting video: 2312/3120 length: 152
Predicting video: 2313/3120 length: 152
Predicting video: 2314/3120 length: 152
Predicting video: 2315/3120 length: 182
Predicting video: 2316/3120 length: 122
Predicting video: 2317/3120 length: 92
Predicting video: 2318/3120 length: 122
Predicting video: 2319/3120 length: 122
Predicting video: 2320/3120 length: 122
Predicting video: 2321/3120 length: 122
Predicting video: 2322/3120 length: 152
Predicting video: 2323/3120 length: 152
Predicting video: 2324/3120 length: 152
Predicting video: 2325/3120 length: 152
Predicting video: 2326/3120 length: 152
Predicting video: 2327/3120 length: 152
Predicting video: 2328/3120 length: 152
Predicting video: 2329/3120 length: 152
Predicting video: 2330/3120 length: 152
Predicting video: 2331/3120 length: 122
Predicting video: 2332/3120 length: 152
Predicting video: 2333/3120 length: 122
Predicting video: 2334/3120 length: 122
Predicting video: 2335/3120 length: 122
Predicting video: 2336/3120 length: 122
P

Predicting video: 2518/3120 length: 152
Predicting video: 2519/3120 length: 122
Predicting video: 2520/3120 length: 122
Predicting video: 2521/3120 length: 152
Predicting video: 2522/3120 length: 122
Predicting video: 2523/3120 length: 122
Predicting video: 2524/3120 length: 122
Predicting video: 2525/3120 length: 152
Predicting video: 2526/3120 length: 152
Predicting video: 2527/3120 length: 152
Predicting video: 2528/3120 length: 152
Predicting video: 2529/3120 length: 152
Predicting video: 2530/3120 length: 152
Predicting video: 2531/3120 length: 122
Predicting video: 2532/3120 length: 122
Predicting video: 2533/3120 length: 122
Predicting video: 2534/3120 length: 152
Predicting video: 2535/3120 length: 122
Predicting video: 2536/3120 length: 122
Predicting video: 2537/3120 length: 122
Predicting video: 2538/3120 length: 122
Predicting video: 2539/3120 length: 152
Predicting video: 2540/3120 length: 122
Predicting video: 2541/3120 length: 152
Predicting video: 2542/3120 length: 152


Predicting video: 2725/3120 length: 122
Predicting video: 2726/3120 length: 92
Predicting video: 2727/3120 length: 92
Predicting video: 2728/3120 length: 122
Predicting video: 2729/3120 length: 92
Predicting video: 2730/3120 length: 92
Predicting video: 2731/3120 length: 92
Predicting video: 2732/3120 length: 92
Predicting video: 2733/3120 length: 92
Predicting video: 2734/3120 length: 92
Predicting video: 2735/3120 length: 92
Predicting video: 2736/3120 length: 92
Predicting video: 2737/3120 length: 92
Predicting video: 2738/3120 length: 122
Predicting video: 2739/3120 length: 92
Predicting video: 2740/3120 length: 122
Predicting video: 2741/3120 length: 122
Predicting video: 2742/3120 length: 122
Predicting video: 2743/3120 length: 122
Predicting video: 2744/3120 length: 122
Predicting video: 2745/3120 length: 122
Predicting video: 2746/3120 length: 92
Predicting video: 2747/3120 length: 92
Predicting video: 2748/3120 length: 92
Predicting video: 2749/3120 length: 92
Predicting video

Predicting video: 2933/3120 length: 92
Predicting video: 2934/3120 length: 122
Predicting video: 2935/3120 length: 92
Predicting video: 2936/3120 length: 92
Predicting video: 2937/3120 length: 92
Predicting video: 2938/3120 length: 92
Predicting video: 2939/3120 length: 92
Predicting video: 2940/3120 length: 122
Predicting video: 2941/3120 length: 122
Predicting video: 2942/3120 length: 122
Predicting video: 2943/3120 length: 122
Predicting video: 2944/3120 length: 122
Predicting video: 2945/3120 length: 122
Predicting video: 2946/3120 length: 152
Predicting video: 2947/3120 length: 92
Predicting video: 2948/3120 length: 122
Predicting video: 2949/3120 length: 92
Predicting video: 2950/3120 length: 122
Predicting video: 2951/3120 length: 152
Predicting video: 2952/3120 length: 152
Predicting video: 2953/3120 length: 182
Predicting video: 2954/3120 length: 152
Predicting video: 2955/3120 length: 152
Predicting video: 2956/3120 length: 122
Predicting video: 2957/3120 length: 122
Predicti

In [25]:
def frame_from_jpeg(path_jpeg):
    img = Image.open(path_jpeg).resize((width, height), Image.ANTIALIAS)
    img = np.array(img)
    return img

In [26]:
"""
for video_ID in dataset["video_ID"].unique()[111:]:
    video_frames = dataset[dataset["video_ID"]==video_ID]
    category_ID = video_frames["category_ID"].unique()[0]
    number_of_frames_in_video = len(video_frames["frames_files"].unique())
    path_to_video = "./data/train_cnn_features/"+category_ID+"/"
    dataset_video_index = list(dataset[dataset["video_ID"]==video_ID].index)
    frames_index = list(dataset[dataset["video_ID"]==video_ID].reset_index(drop=True).index)
    print(f"Predicting video: {video_ID} length: {len(dataset_video_index)}" )
    
    video_frames_jpeg = np.empty((number_of_frames_in_video, width, height, 3), dtype=float)
    i = 0
    for video in video_frames["video_path"].unique():
        print(f"{i+1}/{number_of_frames_in_video}")
        video_frames_jpeg[i] = frame_from_jpeg(video)
        i+=1 
    predictions_video = model_cnn_features.predict(video_frames_jpeg) 
    zeros_to_pad = MAX_SEQ_LENGTH-predictions_video.shape[0]
    predictions_video = np.pad(predictions_video,  [(0, zeros_to_pad), (0, 0)])
    if not os.path.exists(path_to_video):
        os.makedirs(path_to_video)
    np.savez_compressed(path_to_video+video_ID+'.npz', predictions_video)
"""

'\nfor video_ID in dataset["video_ID"].unique()[111:]:\n    video_frames = dataset[dataset["video_ID"]==video_ID]\n    category_ID = video_frames["category_ID"].unique()[0]\n    number_of_frames_in_video = len(video_frames["frames_files"].unique())\n    path_to_video = "./data/train_cnn_features/"+category_ID+"/"\n    dataset_video_index = list(dataset[dataset["video_ID"]==video_ID].index)\n    frames_index = list(dataset[dataset["video_ID"]==video_ID].reset_index(drop=True).index)\n    print(f"Predicting video: {video_ID} length: {len(dataset_video_index)}" )\n    \n    video_frames_jpeg = np.empty((number_of_frames_in_video, width, height, 3), dtype=float)\n    i = 0\n    for video in video_frames["video_path"].unique():\n        print(f"{i+1}/{number_of_frames_in_video}")\n        video_frames_jpeg[i] = frame_from_jpeg(video)\n        i+=1 \n    predictions_video = model_cnn_features.predict(video_frames_jpeg) \n    zeros_to_pad = MAX_SEQ_LENGTH-predictions_video.shape[0]\n    predi